## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-26-13-05-26 +0000,bbc,Budget 2025: Key points at a glance,https://www.bbc.com/news/articles/cj4w44w42j5o...
1,2025-11-26-13-02-12 +0000,startribune,Tough commute in store Wednesday as season’s f...,https://www.startribune.com/tough-commute-in-s...
2,2025-11-26-13-01-20 +0000,bbc,"Isas, cars and pensions - how the Budget affec...",https://www.bbc.com/news/articles/c5y2g2qn0eyo...
3,2025-11-26-13-01-00 +0000,wsj,Gilt Investors Unimpressed by Backloaded Tax M...,https://www.wsj.com/finance/investing/jgb-yiel...
4,2025-11-26-12-57-06 +0000,bbc,Electric vehicle owners to face pay-per-mile tax,https://www.bbc.com/news/articles/c8jw9l7gx92o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2335/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
40,trump,49
53,year,22
42,ukraine,19
162,new,19
43,peace,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
152,2025-11-26-01-40-51 +0000,nypost,Top Trump aide Steve Witkoff coached Russians ...,https://nypost.com/2025/11/25/world-news/top-t...,127
27,2025-11-26-11-52-06 +0000,nypost,Foreigners will have to pay extra $100 to ente...,https://nypost.com/2025/11/26/us-news/trump-ad...,124
265,2025-11-25-20-03-41 +0000,nypost,Trump dispatches top envoys to Russia and Ukra...,https://nypost.com/2025/11/25/us-news/trump-sa...,123
144,2025-11-26-02-22-37 +0000,wapo,"Trump pushes Ukraine, Russia on peace deal, bu...",https://www.washingtonpost.com/world/2025/11/2...,116
116,2025-11-26-07-17-02 +0000,bbc,Trump's envoy to have Ukraine peace plan talks...,https://www.bbc.com/news/articles/cy95jvw57v2o...,116


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
152,127,2025-11-26-01-40-51 +0000,nypost,Top Trump aide Steve Witkoff coached Russians ...,https://nypost.com/2025/11/25/world-news/top-t...
27,75,2025-11-26-11-52-06 +0000,nypost,Foreigners will have to pay extra $100 to ente...,https://nypost.com/2025/11/26/us-news/trump-ad...
39,51,2025-11-26-11-10-35 +0000,nypost,FDA chief promises ‘powerful’ new treatments f...,https://nypost.com/2025/11/26/us-news/fda-chie...
72,49,2025-11-26-10-15-47 +0000,nypost,Hong Kong high rise fire kills four as blaze r...,https://nypost.com/2025/11/26/world-news/hong-...
240,48,2025-11-25-21-14-00 +0000,wsj,At the White House’s annual—and typically stai...,https://www.wsj.com/politics/policy/trump-pard...
317,44,2025-11-25-16-09-36 +0000,nypost,Ukraine peace plan expected to be rejected by ...,https://nypost.com/2025/11/25/world-news/ukrai...
44,44,2025-11-26-11-00-00 +0000,latimes,Chabria: California's first partner pushes to ...,https://www.latimes.com/california/story/2025-...
218,37,2025-11-25-21-59-00 +0000,wsj,Shutdown-delayed data showed American consumer...,https://www.wsj.com/economy/consumers/u-s-reta...
88,36,2025-11-26-09-54-55 +0000,cbc,Israel identifies latest hostage remains retur...,https://www.cbc.ca/news/world/israel-identifie...
103,33,2025-11-26-08-58-31 +0000,wapo,Taiwan plans to spend $40 billion on U.S. arms...,https://www.washingtonpost.com/world/2025/11/2...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
